In [ ]:
# 必要なライブラリ等のインポート
import requests
import json
from datetime import datetime
import pandas as pd

In [ ]:
# SlackのアクセストークンをSlack APIのサイトから取得、きちんと権限を与えているか確認する
access_token = "xoxp-" # ここにアクセストークンを入力する！

In [ ]:
# Slackのチャンネルの履歴を出すAPIを叩く関数、チャンネルIDと最後のトークのタイムスタンプを入れるとJSONで返してくれる
# 一度に1000件しか履歴を取得できない点に注意→gethistdf関数ではループさせて全履歴を取得できるようにしている
def gethistjson(channel_id, latest):
    
    # APIを叩く
    url = "https://slack.com/api/channels.history"
    payload = {
        "token": access_token,
        "channel": channel_id,
        "count": 1000,
        "latest": latest
        }
    response = requests.get(url, params=payload)
    
    # レスポンスされたJSONのmessages部分の階層を出力
    return response.json()["messages"]

In [ ]:
# Slackのパブリックチャンネル一覧を出すAPIを叩く関数、チャンネルIDをarrayで返してくれる、アーカイブされたチャンネルも出力対象
def getchannelarray():
    # APIを叩く
    url = "https://slack.com/api/channels.list"
    payload = {
        "token": access_token
        }
    response = requests.get(url, params=payload)
    
    # チャンネルIDを取り出してarrayに格納
    array = []
    for i in range(len(response.json()["channels"])):
        if i == 0:
            array.append(0)
            array[0] = response.json()["channels"][i]["id"]
        else:
            array.append(response.json()["channels"][i]["id"])
    return array

In [ ]:
# Slackのチャンネルの履歴をすべて出す関数、チャンネルIDを入れるとpandasのData Frameで返してくれる
def gethistdf(channel_id):
    # チャンネルのメッセージ取得
    i = 0
    messages = []
    while True:
        if i == 0:
            # 初回は現在時刻から
            messages.append(i)
            messages[i] = gethistjson(channel_id, datetime.timestamp(datetime.now()))
            df = pd.DataFrame(messages[i])
        else:
            # 次からは前回取得したデータの最後の時間から
            messages.append(i)
            messages[i] = gethistjson(channel_id, float(messages[i - 1][len(messages[i - 1])-1].get("ts")))
            dfc = pd.DataFrame(messages[i])
            # dfにどんどん結合していく
            df = pd.concat([df, dfc])
        
        # もし最後に取得したデータが1000未満ならこれ以上履歴がないという判定
        if len(messages[i]) != 1000:
            # データを合体させたときにindexがおかしくなるからリセット
            df = df.reset_index()
            
            # 行ごとに処理
            for ii in range(len(df)):
                # タイムスタンプ修正
                df['ts'][ii] = datetime.fromtimestamp(float(df['ts'][ii]))
                
                #発言者がBOTの場合、userにbot_idを代入
                if 'bot_id' in df.columns and pd.isnull(df['bot_id'][ii]) is False:
                    df['user'][ii] = df['bot_id'][ii]
                # メッセージIDが欠損しているところは弾く
                df.drop(ii)
            
            # 最後にタイムスタンプをpandasのdatetimeにきちんと修正
            pd.to_datetime(df['ts'])
            
            # データのcolumnの一覧を取得して必要なもの以外はドロップ
            dfcol = df.columns
            dfcol_filter = ['type', 'client_msg_id', 'text', 'ts', 'user', 'reactions']
            for ii in range(len(dfcol)):
                if dfcol[ii] not in dfcol_filter:
                     del df[dfcol[ii]]
           
            # チャンネルIDを追加して全チャンネルのデータを結合してもどのチャンネルからか分かるようにする
            df["channel_id"] = channel_id
            
            # PandasのData Frameで返してあげる
            return df
            break
        else:
            i += 1

In [ ]:
def getallhist():
    for i in range(len(getchannelarray())):
        if i == 0:
            df = gethistdf(getchannelarray()[i])
        else:
            df = pd.concat([df, gethistdf(getchannelarray()[i])])
    # データを合体させたときにindexがおかしくなるからリセット
    df = df.reset_index()
    del df['index']
    return df

In [ ]:
# Slackのチャンネルの履歴のData Frameからリアクションをすべて出す関数、チャンネルの履歴が入ったData Frameを入れるとpandasのData Frameで返してくれる
def getreactiondf(dfr):

    # リアクションの履歴を入れるData Frameを定義
    df = pd.DataFrame(columns =['client_msg_id', 'reaction', 'user'])

    # チャンネルの履歴のData Frameの行ごとに処理
    for i in range(len(dfr)):
        # リアクションのColumnがあるかどうかIf文で確認、ある場合のみ処理
        if 'reactions' in dfr.columns and pd.isnull(dfr['reactions'][i]) is not True and pd.isnull(dfr['client_msg_id'][i]) is not True:

            # リアクションのColumnからリアクションのJSONを抽出
            reaction_raw =  json.loads(json.dumps(dfr['reactions'][i]))

            # 各メッセージのリアクションごとに処理
            for ii in range(len(reaction_raw)):
                
                # 各リアクションのユーザーごとに処理
                users_raw = json.loads(json.dumps(reaction_raw[ii]["users"]))
                for iii in range(len(users_raw)):
                    msg_id = str(dfr['client_msg_id'][i])
                    reaction = reaction_raw[ii]["name"]
                    user = users_raw[iii]
                
                    #print(msg_id + " - " + reaction + ": " + user)
                    df = df.append({'client_msg_id': msg_id, 'reaction': reaction, 'user': user}, ignore_index=True)
                    
    # データを抽出させたときにindexがおかしくなるからリセット
    df = df.reset_index()

    # PandasのData Frameで返してあげる
    return df

### 上記のコードは必ず実行すること！！

In [ ]:
# 全履歴取得
raw_df = getallhist()

In [ ]:
# DFをとりま表示
raw_df

In [ ]:
# 履歴をCSV出力
raw_df.to_csv('SlackChannelHistory - all.csv')

In [ ]:
# リアクション取得
reaction_df = getreactiondf(raw_df)

In [ ]:
# DFをとりま表示
reaction_df

In [ ]:
# 履歴をCSV出力
reaction_df.to_csv('SlackChannelReactionHistory - all.csv')